In [122]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from preprocessor import classify_features

# https://wikidocs.net/22894
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

pd.set_option("display.max_columns", None)


In [123]:
all_data = pd.read_csv("Full-Dataset/processed/merged_data.csv", skipinitialspace=True, index_col="index", engine="python")

In [124]:
drop_columns = ["Unnamed: 0", 'http.request.uri.path', 'http.file_data', "0", "http.request.uri.scheme", "http.request.uri.netloc", "http.referer"]
all_data = all_data.drop([_ for _ in drop_columns if _ in all_data.columns], axis="columns")
all_data = all_data[all_data["frame.time"]!='']
all_data["frame.time"] = pd.to_datetime(all_data["frame.time"])
all_data["tcp.checksum"] = all_data["tcp.checksum"].apply(int, base=16)
all_data["tcp.flags"] = all_data["tcp.flags"].apply(int, base=16)
all_data = all_data.drop(classify_features(all_data)[1], axis="columns")

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_5840\962760474.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data["frame.time"] = pd.to_datetime(all_data["frame.time"])


In [125]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149348 entries, 0 to 93235
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ip.src_host            149348 non-null  object        
 1   frame.time             149348 non-null  datetime64[ns]
 2   tcp.checksum           149348 non-null  int64         
 3   http.response          149348 non-null  float64       
 4   arp.src.proto_ipv4     149348 non-null  object        
 5   http.content_length    149348 non-null  float64       
 6   Attack_type            149348 non-null  object        
 7   arp.dst.proto_ipv4     149348 non-null  object        
 8   tcp.connection.rst     149348 non-null  float64       
 9   arp.hw.size            149348 non-null  float64       
 10  tcp.len                149348 non-null  float64       
 11  tcp.seq                149348 non-null  float64       
 12  tcp.options            149348 non-null  object    

In [126]:
all_data.select_dtypes('object').columns
# http.referer => 0.0 || 0 || 127.0.0.1 => need to be dropped
# ip.src_host, ip.dst_host, arp.src.proto_ipv4, arp.dst.proto_ipv4 need to be vectorized

#  Unknown Columns. Needed to be talked about.
# tcp.options => unknown values. Needed to be tokenized.
# tcp.payload => unknown values. Needed to be tokenized.

Index(['ip.src_host', 'arp.src.proto_ipv4', 'Attack_type',
       'arp.dst.proto_ipv4', 'tcp.options', 'ip.dst_host', 'tcp.payload',
       'http.data.key', 'http.data.value'],
      dtype='object')

In [135]:
all_data[["tcp.options", "tcp.payload"]].nunique()

tcp.options    82640
tcp.payload    52832
dtype: int64

In [127]:
ip_cols = ["ip.src_host", "ip.dst_host", "arp.src.proto_ipv4", "arp.dst.proto_ipv4"]
for ipcol in ip_cols:
    all_data[ipcol] = all_data[ipcol].apply(lambda x: list(map(int, x.split("."))), pd.Int32Dtype)

In [108]:
all_data["tcp.options"].unique()

array(['0101080a9a8a02a2d3a23932', '0101080a9a8a02a3d3a23935',
       '0101080a9a8a031bd3a239b3', ..., '0101080a9ac0f5fa9e4e88bc',
       '0101080a9ac0f5fe9e4e88c1', '0101080a9ac0f6019e4e88c1'],
      dtype=object)

In [89]:
all_data.dtypes

ip.src_host                 object
frame.time                  object
tcp.checksum                object
http.response              float64
arp.src.proto_ipv4          object
http.content_length        float64
Attack_type                 object
arp.dst.proto_ipv4          object
tcp.connection.rst         float64
arp.hw.size                float64
tcp.len                    float64
tcp.seq                    float64
tcp.options                 object
tcp.connection.syn         float64
http.referer                object
ip.dst_host                 object
tcp.flags.ack              float64
tcp.ack_raw                float64
tcp.connection.fin         float64
tcp.dstport                float64
tcp.flags                   object
tcp.ack                    float64
tcp.connection.synack      float64
tcp.payload                 object
tcp.srcport                float64
arp.opcode                 float64
GET                           bool
POST                          bool
TRACE               

In [90]:
all_data.columns

Index(['ip.src_host', 'frame.time', 'tcp.checksum', 'http.response',
       'arp.src.proto_ipv4', 'http.content_length', 'Attack_type',
       'arp.dst.proto_ipv4', 'tcp.connection.rst', 'arp.hw.size', 'tcp.len',
       'tcp.seq', 'tcp.options', 'tcp.connection.syn', 'http.referer',
       'ip.dst_host', 'tcp.flags.ack', 'tcp.ack_raw', 'tcp.connection.fin',
       'tcp.dstport', 'tcp.flags', 'tcp.ack', 'tcp.connection.synack',
       'tcp.payload', 'tcp.srcport', 'arp.opcode', 'GET', 'POST', 'TRACE',
       'HTTP/1.0', 'HTTP/1.1', 'http.request.uri.scheme',
       'http.request.uri.netloc', 'http.data.key', 'http.data.value'],
      dtype='object')

In [91]:
all_data.head()

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_5840\2280888690.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data["frame.time"] = pd.to_datetime(all_data["frame.time"])


ValueError: int() base must be >= 2 and <= 36, or 0

In [23]:
def getRNN_dataset(df):
    columns = ["http.data"]
    columns = [_ for _ in columns if _ in df.columns]
    # return df.loc[:, ["http.data"]].values
    return df[list(columns)]

    
def get_logistic_dataset(df):
    columns = ["http.data"]
    columns = [_ for _ in columns if _ in df.columns]
    return df.drop(columns, axis=1)

In [41]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 데이터를 로드합니다.
df = all_data


# LabelEncoder 클래스를 생성합니다.
encoder = LabelEncoder()
df["Attack_type"] = encoder.fit_transform(df["Attack_type"])
df = df.astype({"http.data.key":"string", "http.data.value":"string"})
df["http.data"] = df["http.data.key"]+'='+df["http.data.value"]
df["http.data"] = df["http.data"].fillna('')
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
splitted_data = train_test_split(df.drop(["Attack_type", "http.data.value", "http.data.key"], axis="columns"), df["Attack_type"], test_size=0.2)
X_train, X_test, y_train, y_test = splitted_data
RNN_X_train, RNN_X_test = [getRNN_dataset(_) for _ in [X_train, X_test]]
logistic_X_train, logistic_X_test = [get_logistic_dataset(_) for _ in [X_train, X_test]]

In [42]:
encoder.classes_

array([0, 1, 2, 3], dtype=int64)

In [ ]:
# https://wikidocs.net/22894
tokenizer = Tokenizer()
tokenizer.fit_on_texts(RNN_X_train["http.data"])
RNN_X_train_encoded = tokenizer.texts_to_sequences(RNN_X_train["http.data"])

In [45]:
RNN_X_train_encoded[:5]


[[], [1, 1], [], [], [2, 3]]

In [47]:
word_to_index = tokenizer.word_index
tokenizer.word_index

{'login': 1,
 'username': 2,
 'admin': 3,
 'password': 4,
 '0': 5,
 '00': 6,
 'chr': 7,
 'id': 8,
 'submit': 9,
 '113': 10,
 'submitid': 11,
 'and': 12,
 'char': 13,
 'select': 14,
 'from': 15,
 '5': 16,
 '1': 17,
 'case': 18,
 'when': 19,
 'then': 20,
 'else': 21,
 'end': 22,
 '98': 23,
 '107': 24,
 '58': 25,
 'sleep': 26,
 'dual': 27,
 '120': 28,
 '122': 29,
 'order': 30,
 'by': 31,
 '2': 32,
 '106': 33,
 '9': 34,
 '118': 35,
 '8': 36,
 '57': 37,
 '7': 38,
 '54': 39,
 '55': 40,
 '59': 41,
 '3': 42,
 'pg': 43,
 '4': 44,
 'waitfor': 45,
 'delay': 46,
 "'0": 47,
 "5'": 48,
 '53': 49,
 'dbms': 50,
 'pipe': 51,
 'receive': 52,
 'message': 53,
 '112': 54,
 'concat': 55,
 'extractvalue': 56,
 '0x5c': 57,
 'elt': 58,
 'in': 59,
 '49': 60,
 '48': 61,
 'upper': 62,
 'xmltype': 63,
 '60': 64,
 '62': 65,
 'cast': 66,
 'text': 67,
 'as': 68,
 'numeric': 69,
 "59'": 70,
 "4'": 71,
 "55'": 72,
 "7'": 73,
 "54'": 74,
 "53'": 75,
 "58'": 76,
 "57'": 77,
 "3'": 78,
 "2'": 79,
 "8'": 80,
 "9'": 81,
 "'

In [48]:
threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


등장 빈도가 1번 이하인 희귀 단어의 수: 3415
단어 집합(vocabulary)에서 희귀 단어의 비율: 73.56742783283067
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8558370558761834


In [53]:
vocab_size = len(word_to_index) + 1
max_length_of_http_data = max(len(sample) for sample in RNN_X_train_encoded)
avg_length_of_http_data = (sum(map(len, RNN_X_train_encoded))/len(RNN_X_train_encoded))
print('단어 집합의 크기: {}'.format((vocab_size)))

단어 집합의 크기: 4643


In [54]:
RNN_X_train_padded = pad_sequences(RNN_X_train_encoded, maxlen = max_length_of_http_data)
print("훈련 데이터의 크기(shape):", RNN_X_train_padded.shape)

훈련 데이터의 크기(shape): (119478, 51)


In [64]:
def RNN_padding(tokenizer:Tokenizer, series, max_len):
    return pad_sequences(tokenizer.texts_to_sequences(series), maxlen=max_len)

In [27]:
RNN_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119478 entries, 95791 to 8026
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   http.data  63821 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


In [59]:
# RNN을 사용하여 요청의 URI 키와 값을 분류하는 모델을 학습합니다.
rnn_model = tf.keras.Sequential([
  layers.Embedding(vocab_size, 128),
  layers.SimpleRNN(128),
  layers.Dense(1, activation="sigmoid")
])

In [50]:
rnn_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, None, 128)         128000    
                                                                 
 lstm_16 (LSTM)              (None, 128)               131584    
                                                                 
 dense_14 (Dense)            (None, 10)                1290      
                                                                 
Total params: 260,874
Trainable params: 260,874
Non-trainable params: 0
_________________________________________________________________


In [63]:
rnn_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
rnn_model.fit(RNN_X_train_padded, y_train, epochs=10)

Epoch 1/10


c:\Users\ParkJaehyung\OneDrive\바탕 화면\고려대학교\2학년 1학기\기계학습개론\KU-ML-EndTerm-project\myvenv\Lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2460/3734 [==================>...........] - ETA: 23s - loss: 0.0000e+00 - accuracy: 0.7918

KeyboardInterrupt: 

In [ ]:
rnn_model.evaluate(RNN_padding(tokenizer,RNN_X_test,max_length_of_http_data), y_test)

In [66]:
# 로지스틱 회귀를 사용하여 나머지 데이터를 분류하는 모델을 학습합니다.
logistic_regression_model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(4, activation="softmax")
])

In [67]:
logistic_regression_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
logistic_regression_model.fit(X_train, y_train, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# 두 모델의 예측을 결합하여 최종 예측을 만듭니다.
predictions = rnn_model.predict(RNN_padding(tokenizer,RNN_X_test,max_length_of_http_data)) * 0.5 + logistic_regression_model.predict(logistic_X_test) * 0.5

# 모델의 성능을 평가합니다.
accuracy = tf.metrics.accuracy(labels=y_test, predictions=predictions)
print("Accuracy:", accuracy)